# **CUHK-STAT3009** Notebook - Correlation-based Recommender Systems


In [ ]:
# Load and pro-processed dataset
import numpy as np
import pandas as pd

## Upload Netflix dataset in CUHK-STAT3009 Github repo

train_url = "https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/train.csv"
test_url = "https://raw.githubusercontent.com/statmlben/CUHK-STAT3009/main/dataset/test.csv"

dtrain = pd.read_csv(train_url)
dtest = pd.read_csv(test_url)

train_rating = dtrain['rating'].values
train_rating = np.array(train_rating, dtype=float)
train_pair = dtrain[['user_id', 'movie_id']].values

test_rating = dtest['rating'].values
test_rating = np.array(test_rating, dtype=float)
test_pair = dtest[['user_id', 'movie_id']].values

n_user = max( max(train_pair[:,0]), max(test_pair[:,0]) ) + 1
n_item = max( max(train_pair[:,1]), max(test_pair[:,1]) ) + 1

## Implement Correlation-based (user-based) recommender systems

- Method 1: store distance matrix

  - Steps
    - develop `user_co_rating` function
    - define `user_d` function
    - define `user_d_mat`
    - make prediction based on Algo1

- Method 2: without store distance matrix

  - Steps
    - Loop over $u$ then loop over $i \in \mathcal{I}_u$
    - Compute distance for user u in the loop over u
    - find a valid neighbor in the loop *over* $i$


- Note
  - time-space trade-off: [how to profiling and timing Code](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html)?

### `user_co_rating`: find co-rating items for two users

- Structure:

  - **Input**: pair of users $(u,v)$; training set 
  - **Output**: co-rating items and their ratings

- Steps:

  - Find all indices for $u$ and $v$
  - Find co-rated items
  - Find indices for co-rated items for $u$ and $v$ separately

- Tips:

  - save indices for all users/items

In [ ]:
# save indices for all users/items
index_item = [np.where(train_pair[:,1] == i)[0] for i in range(n_item)]
index_user = [np.where(train_pair[:,0] == u)[0] for u in range(n_user)]

In [ ]:
# test index sets
train_pair[index_user[0]].T

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [2720, 1935, 3126, 3386, 1118, 1686,  151, 1456, 2893,  571,  712,
         586, 3148, 2306, 3295,  395, 2198, 1546, 1164, 3256,  426,  977,
        2885, 1019, 3487, 1292, 3383, 1577, 2790, 2337, 2345,  945, 1428,
        1306, 2977, 2531, 1133, 1778, 1473, 3276, 2157, 1501,  993, 3516,
        2716, 2497,  453,  916, 2554, 3114, 1790, 1931, 2505, 2922, 1734,
        1395, 3007, 1504,  133, 3058, 1423,  183,  634, 1333,  132,  467,
         285, 1555, 224

In [ ]:
def user_co_rating(index_u,index_v,train_pair,train_rating):
    """
    Find co-rated items of two users.

    Input
    -----
    index_u: index set of rated records for user u
    index_v: index set of rated records for user v
    train_pair: user-item pairs in training set
    train_rating: ratings for `train_pair`

    Output
    ------
    indices for co-rated items by users u and v
    """
    ## Step 1: find co-rated item

    # find items for u and v
    item_u = train_pair[index_u][:,1]
    item_v = train_pair[index_v][:,1]
    # find co-rating items by `set`
    item_co = list(set(item_u).intersection(set(item_v)))
    if len(item_co) == 0:
        return None, None
    else:
        ## Step 2: indices for co-rated items for both users
        
        # find the co-rating vectors by using `np.isin`
        ind_co_u = np.where(np.isin(item_u, item_co))[0]
        ind_co_v = np.where(np.isin(item_v, item_co))[0]

        # Make sure the indices for both users are corresponding to the same items
        ind_co_u = ind_co_u[np.argsort(item_u[ind_co_u])]
        ind_co_v = ind_co_v[np.argsort(item_v[ind_co_v])]

        ## alternative way to find the valid indices
        # ind_co_u = [np.where(item_u == item_co_tmp)[0][0] for item_co_tmp in item_co]
        # ind_co_v = [np.where(item_v == item_co_tmp)[0][0] for item_co_tmp in item_co]
        return index_u[ind_co_u], index_v[ind_co_v]

In [ ]:
## test function
ind_co_u, ind_co_v = user_co_rating(index_user[0],index_user[1],train_pair,train_rating)
train_pair[ind_co_u].T

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [ 133,  229,  571,  712,  916, 1118, 1133, 1306, 1448, 1456, 1501,
        1504, 1778, 2198, 2497, 2505, 2893, 2922, 2977, 3058, 3114, 3386,
        3543]])

In [ ]:
train_pair[ind_co_v].T

array([[   1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1],
       [ 133,  229,  571,  712,  916, 1118, 1133, 1306, 1448, 1456, 1501,
        1504, 1778, 2198, 2497, 2505, 2893, 2922, 2977, 3058, 3114, 3386,
        3543]])

### Define `user_d` function

- Structure:
  - Input: pair of ind of users (𝑢,𝑣); training set
  - Output: co-rating items and their ratings

- Steps:
  - Find indices for co-rated items for 𝑢 and 𝑣 by `user_co_rating`
  - Compute distance



In [ ]:
from scipy import spatial

def user_d(index_u,index_v,train_pair,train_rating, distance='euclidean'):
    """
    Find co-rated items of two users.

    Input
    -----
    index_u: index set of rated records for user u
    index_v: index set of rated records for user v
    train_pair: user-item pairs in training set
    train_rating: ratings for `train_pair`

    Output
    ------
    distance btw users u and v
    """

    ind_co_u, ind_co_v = user_co_rating(index_u,index_v,train_pair,train_rating)
    if (ind_co_u is None) or (ind_co_v is None):
      return 0.
    rating_co_u, rating_co_v = train_rating[ind_co_u], train_rating[ind_co_v]
    if distance == 'euclidean':
      return np.sqrt(np.sum((rating_co_u - rating_co_v)**2)) + 1e-4
    elif distance == 'cosine':
      return spatial.distance.cosine(rating_co_u, rating_co_v) + 1e-4
    elif distance == 'correlation':
      return np.corrcoef(rating_co_u, rating_co_v) + 1e-4
    else:
      raise Exception("Sorry, distance must be euclidean, cosine, or correlation") 

In [ ]:
## test function
user_d(index_user[0],index_user[1],train_pair,train_rating)

4.69041575982343

In [ ]:
## test (u,v) = (0,4); None co-rated items
print(user_co_rating(index_user[0],index_user[4],train_pair,train_rating))
print(user_d(index_user[0],index_user[4],train_pair,train_rating))

(None, None)
0.0


### Define `user_d_mat` function

- Structure:
  - Input: training set
  - Output: distance matrix for all users

- Steps:
  - call `user_d` in a pairwise look

- Note:
  - save the distance matrix by `scipy.sparse.lil_matrix`

In [ ]:
from scipy.sparse import lil_matrix

def user_d_mat(train_pair, train_rating, distance='euclidean'):
    ## Compute the similarity matrix for all users
    # sparse matrix 
    D = lil_matrix((n_user, n_user))
    # pairwise loop over users
    for u in range(n_user):
        if u%500 == 0:
            print('dis-vec for user_id %d' %u)
        for v in range(u):
          # we could cut more
            if (len(index_user[u]) == 0) or (len(index_user[v]) == 0):
                continue
            ## to make a difference to zero
            D[u,v] = user_d(index_user[u],index_user[v],train_pair,train_rating,distance)
    return D + D.T

In [ ]:
D_mat = user_d_mat(train_pair, train_rating)

dis-vec for user_id 0
dis-vec for user_id 500
dis-vec for user_id 1000
dis-vec for user_id 1500


In [ ]:
D_mat.nnz
print('sparseness: %.3f' %(1-D_mat.nnz/(n_user**2)))

sparseness: 0.621


In [ ]:
D_mat[0,1]

4.69041575982343

### Make prediction based on `D_mat`

- Steps
  - Loop over $(u,i) \in \Omega^{te}$
  - Find a valid top-k neighbor with C1 & C2
  - Average over the neighbor

In [ ]:
glb_mean = train_rating.mean()
pred_rating = np.zeros_like(test_rating)

# loop over test records
for j in range(len(test_pair)):
    ## find user_id and item_id for each record in test_pair
    user_tmp, item_tmp = test_pair[j,0], test_pair[j,1]
    
    # C1: find which users rated `item_tmp`
    ## find index of all records rated item_tmp
    index_tmp = index_item[item_tmp]
    ## from `index_tmp`, we find the users rated `item_tmp`, and its ratings
    rated_users = train_pair[index_tmp][:,0]
    rated_ratings = train_rating[index_tmp]
    # C2: compute the distance of `rated_users`
    D_tmp = D_mat[user_tmp, rated_users].toarray()[0]
    ## remove dissimilar users: 0 means no corated items for two users
    rated_users = rated_users[D_tmp>0]
    rated_ratings = rated_ratings[D_tmp>0]
    D_tmp = D_tmp[D_tmp>0]
    if (len(D_tmp) == 0):
        # if no rated users or no similar users, predicted by user_mean
        ## find the records for `user_tmp`
        index_user_tmp = index_user[user_tmp]
        # if no records for user_tmp
        ## predict by glb mean
        if len(index_user_tmp) == 0:
            pred_rating[j] = glb_mean
        else:
            pred_rating[j] = train_rating[index_user_tmp].mean()
    else:
        # if we do find similar co-rated user
        ## predict by average of top-10 users
        
        # find top 5 rated-users
        top_index = np.argsort(D_tmp)[:50]
        pred_rating[j] = np.mean(rated_ratings[top_index])

In [ ]:
## RMSE for correlation based RS
rmse_crs = np.sqrt(np.mean((pred_rating - test_rating)**2))
print('RMSE for correlation-base RS: %.3f' %rmse_crs)

RMSE for correlation-base RS: 1.042


## Prediction without distance matrix

- Loop over $u$ then loop over $i \in \mathcal{I}_u$
- Compute distance for user u in the loop over u
- find a valid neighbor in the loop *over* $i$

In [ ]:
pred_rating = np.zeros_like(test_rating)
glb_mean = train_rating.mean()

# loop over user
for user_tmp in range(n_user):
    if user_tmp%500 == 0:
        print('predict for user_id %d' %user_tmp)
    # find all records for user_tmp in test_pair
    # which we want to make prediction in this loop
    index_user_tmp = np.where(test_pair[:,0] == user_tmp)[0]
    if len(index_user_tmp) == 0:
        # no record to predict for this user, skip this user
        continue
    # the index of records that `user_tmp` rated in training set
    train_index_user_tmp = index_user[user_tmp]
    # compute weights of `user_tmp` across others
    d_all = [user_d(index_user[user_tmp],index_user[v],train_pair,train_rating) for v in range(n_user)]
    d_all = np.array(d_all)
    # loop over items in test set of `user_tmp`
    for record in index_user_tmp:
        item_tmp = test_pair[record,1]
        # find co-rated users on `item_tmp`
        ## find the index of `item_tmp`
        index_item_tmp = index_item[item_tmp]
        ## find the rated_users of `item_tmp`
        rated_users = train_pair[index_item_tmp][:,0]
        rated_ratings = train_rating[index_item_tmp]
        d_rated = d_all[rated_users]
        ## remove the users with 0 weights, which means no shared record with user u
        rated_users = rated_users[d_rated>0]
        rated_ratings = rated_ratings[d_rated>0]
        d_rated = d_rated[d_rated>0]
        ## find the weight of rated users
        if len(rated_users) == 0:
            # if no rated users, then predict by user-mean
            if len(train_index_user_tmp) == 0:
                pred_rating[record] = glb_mean
            else:
                pred_rating[record] = train_rating[train_index_user_tmp].mean()
        else:
            top_index = np.argsort(d_rated)[:20]
            pred_rating[record] = np.mean(rated_ratings[top_index])

predict for user_id 0
predict for user_id 500
predict for user_id 1000
predict for user_id 1500


In [ ]:
## RMSE for correlation based RS
rmse_crs = np.sqrt(np.mean((pred_rating - test_rating)**2))
print('RMSE for correlation-base RS: %.3f' %rmse_crs)

RMSE for correlation-base RS: 1.044


## Package user-based correlation RS as a Python function

In [ ]:
def cor_rs_user(train_pair, train_rating, test_pair):
    # compute basic quatities
    n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1
    index_item = [np.where(train_pair[:,1] == i)[0] for i in range(n_item)]
    index_user = [np.where(train_pair[:,0] == u)[0] for u in range(n_user)]

    pred_rating = np.zeros(len(test_pair))
    # compute glb mean
    glb_mean = train_rating.mean()

    # loop over user
    for user_tmp in range(n_user):
        if user_tmp%500 == 0:
            print('predict for user_id %d' %user_tmp)
        # find all records for user_tmp in test_pair
        # which we want to make prediction in this loop
        index_user_tmp = np.where(test_pair[:,0] == user_tmp)[0]
        if len(index_user_tmp) == 0:
            # no record to predict for this user, skip this user
            continue
        # the index of records that `user_tmp` rated in training set
        train_index_user_tmp = index_user[user_tmp]
        # compute weights of `user_tmp` across others
        d_all = [user_d(index_user[user_tmp],index_user[v],train_pair,train_rating) for v in range(n_user)]
        d_all = np.array(d_all)
        # loop over items in test set of `user_tmp`
        for record in index_user_tmp:
            item_tmp = test_pair[record,1]
            # find co-rated users on `item_tmp`
            ## find the index of `item_tmp`
            index_item_tmp = index_item[item_tmp]
            ## find the rated_users of `item_tmp`
            rated_users = train_pair[index_item_tmp][:,0]
            rated_ratings = train_rating[index_item_tmp]
            d_rated = d_all[rated_users]
            ## remove the users with 0 weights, which means no shared record with user u
            rated_users = rated_users[d_rated>0]
            rated_ratings = rated_ratings[d_rated>0]
            d_rated = d_rated[d_rated>0]
            ## find the weight of rated users
            if len(rated_users) == 0:
                # if no rated users, then predict by user-mean
                if len(train_index_user_tmp) == 0:
                    pred_rating[record] = glb_mean
                else:
                    pred_rating[record] = train_rating[train_index_user_tmp].mean()
            else:
                top_index = np.argsort(d_rated)[:20]
                pred_rating[record] = np.mean(rated_ratings[top_index])
    return pred_rating

In [ ]:
## test function

pred_rating = cor_rs_user(train_pair, train_rating, test_pair)

predict for user_id 0
predict for user_id 500
predict for user_id 1000
predict for user_id 1500


In [ ]:
## RMSE for correlation based RS
rmse_crs = np.sqrt(np.mean((pred_rating - test_rating)**2))
print('RMSE for correlation-base RS: %.3f' %rmse_crs)

RMSE for correlation-base RS: 1.044


## Sequential models: baseline + correlation RS

- fit `user-mean` and `item-mean`
- then use `correlation-based` RS

In [ ]:
## recall baseline methods

def user_mean(train_pair, train_rating, test_pair):
    n_user = max(train_pair[:,0].max(), test_pair[:,0].max())+1
    pred = np.zeros(len(test_pair))
    glb_mean_value = train_rating.mean()
    for u in range(n_user):
        # find the index for both train and test for user_id = u
        ind_test = np.where(test_pair[:,0] == u)[0]
        ind_train = np.where(train_pair[:,0] == u)[0]
        if len(ind_test) == 0:
            continue
        if len(ind_train) < 3:
            pred[ind_test] = glb_mean_value
        else:
            # predict as user average
            pred[ind_test] = train_rating[ind_train].mean()
    return pred

def item_mean(train_pair, train_rating, test_pair):
    n_item = max(train_pair[:,1].max(), test_pair[:,1].max())+1
    pred = np.zeros(len(test_pair))
    glb_mean_value = train_rating.mean()
    for i in range(n_item):
        # find the index for both train and test for item_id = i
        ind_test = np.where(test_pair[:,1] == i)[0]
        ind_train = np.where(train_pair[:,1] == i)[0]
        if len(ind_test) == 0:
            continue
        if len(ind_train) < 3:
            pred[ind_test] = glb_mean_value
        else:
            # predict as user average
            pred[ind_test] = train_rating[ind_train].mean()
    return pred

def rmse(true_rating, pred_rating):
  return np.sqrt(np.mean((true_rating - pred_rating)**2))

In [ ]:
pred_rating = user_mean(train_pair, train_rating, test_pair)
res_rating = train_rating - user_mean(train_pair, train_rating, train_pair)

print('rmse for user mean: %.3f' %rmse(test_rating, pred_rating))

rmse for user mean: 1.013


In [ ]:
## compute the residual rating
## test_pair -> train_pair

pred_res_item = user_mean(train_pair, res_rating, test_pair)

res_rating = res_rating - item_mean(train_pair, res_rating, train_pair)
pred_rating = pred_rating + pred_res_item
print('rmse for user mean: %.3f' %rmse(test_rating, pred_rating))

rmse for user mean: 0.964


In [ ]:
pred_res_corr = cor_rs_user(train_pair, res_rating, test_pair)
pred_rating = pred_rating + pred_res_corr

predict for user_id 0
predict for user_id 500
predict for user_id 1000
predict for user_id 1500


In [ ]:
print('rmse for user mean: %.3f' %rmse(test_rating, pred_rating))

rmse for user mean: 0.984


## To-do list

- **STAT**
  - [ ] Understand the motivation of correlation based methods
  - [ ] strength and weakness for each method
  - [ ] Understand various distance

- **Code**

  - [ ] implement `item_co_rating`, `item_d`
  - [ ] explore [`scipy.sparse`](https://docs.scipy.org/doc/scipy/reference/sparse.html)
  - [ ] implement Algos 3 and 4 for item-based correlation RS